In [4]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": 52.0499,
	"longitude": 5.07391,
	"start_date": "2016-05-01",
	"end_date": "2020-08-31",
	"hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "apparent_temperature", "precipitation", "rain", "snowfall", "snow_depth", "pressure_msl", "surface_pressure", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high", "shortwave_radiation_instant", "direct_radiation_instant", "diffuse_radiation_instant", "direct_normal_irradiance_instant", "wind_speed_10m", "wind_direction_10m"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
hourly_apparent_temperature = hourly.Variables(3).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(4).ValuesAsNumpy()
hourly_rain = hourly.Variables(5).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(6).ValuesAsNumpy()
hourly_snow_depth = hourly.Variables(7).ValuesAsNumpy()
hourly_pressure_msl = hourly.Variables(8).ValuesAsNumpy()
hourly_surface_pressure = hourly.Variables(9).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(10).ValuesAsNumpy()
hourly_cloud_cover_low = hourly.Variables(11).ValuesAsNumpy()
hourly_cloud_cover_mid = hourly.Variables(12).ValuesAsNumpy()
hourly_cloud_cover_high = hourly.Variables(13).ValuesAsNumpy()
hourly_shortwave_radiation_instant = hourly.Variables(14).ValuesAsNumpy()
hourly_direct_radiation_instant = hourly.Variables(15).ValuesAsNumpy()
hourly_diffuse_radiation_instant = hourly.Variables(16).ValuesAsNumpy()
hourly_direct_normal_irradiance_instant = hourly.Variables(17).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(18).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(19).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["pressure_msl"] = hourly_pressure_msl
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["shortwave_radiation_instant"] = hourly_shortwave_radiation_instant
hourly_data["diffuse_radiation_instant"] = hourly_diffuse_radiation_instant
hourly_data["direct_normal_irradiance_instant"] = hourly_direct_normal_irradiance_instant
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_direction_10m"] = hourly_wind_direction_10m

hourly_dataframe = pd.DataFrame(data = hourly_data)
hourly_dataframe = hourly_dataframe.set_index('date')
hourly_dataframe.head()


Coordinates 52.056236267089844°N 5.128676414489746°E
Elevation 4.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,temperature_2m,relative_humidity_2m,pressure_msl,cloud_cover,shortwave_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,wind_speed_10m,wind_direction_10m
date,,,,,,,,,
2016-05-01 00:00:00+00:00,3.6805,89.285660,1025.300049,0.0,0.0,0.0,0.0,11.165805,358.152435
2016-05-01 01:00:00+00:00,3.1805,90.854256,1025.800049,0.0,0.0,0.0,0.0,9.028754,4.573841
2016-05-01 02:00:00+00:00,2.8305,91.482162,1026.199951,2.1,0.0,0.0,0.0,10.080000,360.000000
2016-05-01 03:00:00+00:00,2.6305,91.797310,1026.199951,10.2,0.0,0.0,0.0,11.885453,1.735665
2016-05-01 04:00:00+00:00,2.5805,91.794098,1026.599976,18.0,0.0,0.0,0.0,9.470120,8.746089


In [5]:
import pickle
hourly_dataframe.to_pickle('openmeteo.pickle')